In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('cluster').getOrCreate()

24/04/07 09:34:16 WARN Utils: Your hostname, Luiss-Mac-mini.local resolves to a loopback address: 127.0.0.1; using 192.168.100.206 instead (on interface en1)
24/04/07 09:34:16 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/07 09:34:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/04/07 09:34:17 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/04/07 09:34:17 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
24/04/07 09:34:17 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
24/04/07 09:34:17 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
24/04/07 09:34:17 WARN Utils: Service 'SparkUI' could not bind on port 4044. Attempti

In [12]:
from pyspark.ml.clustering import KMeans

In [13]:
dataset = spark.read.format('libsvm').load('sample_kmeans_data.txt')

24/04/07 09:37:13 WARN LibSVMFileFormat: 'numFeatures' option not specified, determining the number of features by going though the input. If you know the number in advance, please specify it via 'numFeatures' option to avoid the extra scan.


In [14]:
dataset.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|           (3,[],[])|
|  1.0|(3,[0,1,2],[0.1,0...|
|  2.0|(3,[0,1,2],[0.2,0...|
|  3.0|(3,[0,1,2],[9.0,9...|
|  4.0|(3,[0,1,2],[9.1,9...|
|  5.0|(3,[0,1,2],[9.2,9...|
+-----+--------------------+



In [15]:
final_data = dataset.select('features')

In [16]:
final_data.show()

+--------------------+
|            features|
+--------------------+
|           (3,[],[])|
|(3,[0,1,2],[0.1,0...|
|(3,[0,1,2],[0.2,0...|
|(3,[0,1,2],[9.0,9...|
|(3,[0,1,2],[9.1,9...|
|(3,[0,1,2],[9.2,9...|
+--------------------+



In [30]:
kmeans = KMeans().setK(3).setSeed(1)

In [31]:
model = kmeans.fit(final_data)
predictions = model.transform(final_data)

In [32]:
from pyspark.ml.evaluation import ClusteringEvaluator

In [33]:
evaluator = ClusteringEvaluator()
silhouette = evaluator.evaluate(predictions)

In [34]:
print("Silhouette with squared euclidean distance = " + str(silhouette))

Silhouette with squared euclidean distance = 0.6248737134600261


In [35]:
centers = model.clusterCenters()

In [36]:
centers

[array([9.1, 9.1, 9.1]), array([0.05, 0.05, 0.05]), array([0.2, 0.2, 0.2])]

In [37]:
predictions.show()

+--------------------+----------+
|            features|prediction|
+--------------------+----------+
|           (3,[],[])|         1|
|(3,[0,1,2],[0.1,0...|         1|
|(3,[0,1,2],[0.2,0...|         2|
|(3,[0,1,2],[9.0,9...|         0|
|(3,[0,1,2],[9.1,9...|         0|
|(3,[0,1,2],[9.2,9...|         0|
+--------------------+----------+

